In [1]:
import json
import numpy as np
import os
import re
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Cm, Pt, Inches
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE
from h2ogpte import H2OGPTE
from mediawikiapi import MediaWikiAPI
from tqdm import tqdm
with open('secrets.txt') as f:
    api = f.read()
    f.close()



In [2]:
client = H2OGPTE(
    address="https://h2ogpte.genai.h2o.ai",
    api_key=api
)

user_query = 'Create a presentation on the Pokemon franchise'

In [3]:
def trim(lst, retain=30):
    '''
    Trims a list. This function was originally used to permute before trimming, but\
    now that functionality is removed, so it appears rather redundant.
    '''
    _ = lst.copy()
    
    return _[0:retain]



def try_and_parse(user_query, function, failed=0, markdown=False):
    '''
    Accepts a function and user_query, an input. Evaluates function(user_query) and 
    converts string output (usually a reply from an llm) into a json value. Use markdown=True
    if the json value is contained within a code chunk.
    '''
    chosen = function(user_query)
    try:
        if not markdown:
            topics = json.loads(chosen.content)
        else:
            print(chosen.content)
            pattern = r'^```(?:\w+)?\s*\n(.*?)(?=^```)```'
            result = re.findall(pattern, chosen.content, re.DOTALL | re.MULTILINE)[0].strip() 
            #print(result)
            topics = json.loads(result)
            
        return topics
    except Exception as e:
        failed+=1
        print(failed)
        print(e)# CHANGE TO LOGGING STATEMENT
        return try_and_parse(user_query, function, failed=failed, markdown=markdown)


## Step 1. What comes to mind when you think about xyz?

In [4]:
search = lambda user_query: client.answer_question(
    question=user_query,
    system_prompt="""You are an assistant whose task is to perform Wikipedia searches on a specific topic.\
    The user is interested to create a presentation about a topic of interest.\
    Reply with at least one corresponding Wikipedia query as an array in JSON format.\
    Only reply with the JSON array and nothing else. Here are some examples.
    Example 1.
    User: Create a presentation on the book, Baby Rudin.
    Assistant: ["Real analysis", "Mathematical Analysis"]

    Example 2.
    User: I want to create a ppt about Milk.
    Assistant: ["Milk", "Plant Milk", "Goat's Milk", "Cow Milk", "Almond Milk"]
    """,
    llm='mistralai/Mixtral-8x7B-Instruct-v0.1'
)
searched = try_and_parse(user_query, search)
searched

['Pokémon franchise',
 'Pokémon media',
 'Pokémon video games',
 'Pokémon anime and manga',
 'Pokémon Trading Card Game',
 'List of Pokémon species',
 'List of Pokémon media',
 'List of Pokémon video games',
 'List of Pokémon episodes',
 'List of Pokémon characters',
 'List of Pokémon manga']

## Step 2. Search Wikipedia

In [5]:
import random
wiki = MediaWikiAPI()

articles = list(
    set(
        
        [i for j in [wiki.search(cat, results=5) for cat in searched] for i in j]
        
    )
) # remove duplicates with set(list())
random.shuffle(articles) # random shuffle



In [6]:
snippet = trim(
    list(map(lambda x: wiki.summary(x, auto_suggest=False, sentences=1), articles))
)

snippet_ = snippet.copy()
i = 0
for string in snippet:
    string = f"{i}. {string}"
    snippet[i] = string
    i+=1
snippet_text = "\n\n".join(snippet)
print(snippet_text)

0. Pokémon (originally "Pocket Monsters") is a series of role-playing video games developed by Game Freak and published by Nintendo and The Pokémon Company.

1. Pokémon, known in Japan as Pocket Monsters (ポケットモンスター, Poketto Monsutā), is a Japanese anime television series produced by animation studio OLM for TV Tokyo.

2. The Pokémon Company (株式会社ポケモン, Kabushiki Gaisha Pokemon, TPC) is a Japanese company responsible for brand management, production, publishing, marketing, and licensing of the Pokémon franchise, which consists of video games, a trading card game, anime television series, films, manga, home entertainment products, merchandise, and other ventures.

3. The seventh generation (Generation VII) of the Pokémon franchise features 86 fictional species of collectible creatures called "Pokémon" introduced to the core video game series in the 2016 Nintendo 3DS games Pokémon Sun and Moon and the 2017 3DS games Pokémon Ultra Sun and Ultra Moon.

4. The first generation (generation I) 

## Step 3. "Brainstorm" and filter Wikipedia searches for useful ones
Chain-of-thought prompting
https://www.promptingguide.ai/techniques/cot

In [7]:
choose_topics = lambda user_query: client.answer_question(
        question=f"""{user_query}
        Referring to the list of wikipedia entries you have been provided, decide on which topics are useful for the presentation. For each entry, explain, in a few words,\
        whether you think an entry is useful or not and why.
        After that, generate a code chunk. Within the code chunk is an array of integers in JSON, and these integers correspond to the topics you think are useful.
        For each line, think about its usefulness/relevance and make some short deductions, then conclude whether it is useful or not useful.
        Please keep strictly to the format in the following example:
        0. - Sugar irrelevant to Jesus Christ, therefore not useful.
        1. - Christianity is about the topic of Jesus Christ, thus Useful.
        2. - Protestants follow Jesus Christ, therefore useful.
        ```
        [1, 2]
        ```
        """,
        system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation.\
        Here are a list of wikipedia entries, starting from the 0th entry, that may or may not be related to the topic at hand:
        {snippet_text}
        """,
        llm='gpt-4-1106-preview' # only instance of gpt4 usage. need this for the big brain.
    )

topics = try_and_parse(user_query, choose_topics, markdown=True)
print(topics)

0. - Introduction to the Pokémon franchise, including its development and publishers, therefore useful.
1. - Details about the Pokémon anime series, relevant for understanding the multimedia aspect of the franchise, therefore useful.
2. - Information on the company managing the Pokémon brand, relevant for corporate and business aspects of the franchise, therefore useful.
3. - Specific information about the seventh generation of Pokémon games, useful for discussing the evolution of the game series.
4. - Information on the first generation of Pokémon, important for historical context and franchise origins, therefore useful.
5. - Manga adaptations add to the multimedia scope of the franchise, therefore useful.
6. - General video game sales data, not specifically about Pokémon, therefore not useful.
7. - Reiterates the development and publishing of the Pokémon video games, redundant with entry 0, therefore not useful.
8. - Specific information about the fourth generation of Pokémon games, 

In [8]:
articles

['List of Pokémon video games',
 'List of Pokémon episodes (seasons 1–13)',
 'The Pokémon Company',
 'List of generation VII Pokémon',
 'List of generation I Pokémon',
 'List of Pokémon manga',
 'List of best-selling video games',
 'Pokémon (video game series)',
 'List of generation IV Pokémon',
 'Pokémon Adventures',
 'Pokémon World Championships',
 'Pokémon Diamond and Pearl',
 'Lists of Pokémon episodes',
 'Pokémon: Adventures in the Orange Islands',
 'List of generation II Pokémon',
 'List of Pokémon episodes (seasons 14–present)',
 'Pokémon Crystal',
 'Pokémon: Master Quest',
 'List of Pokémon theme songs',
 'Pokémon Trading Card Game (video game)',
 'Pokémon Pocket Monsters',
 'List of Pokémon characters',
 'List of Pokémon volumes',
 'Pokémon Red, Blue, and Yellow',
 'Pokémon',
 'Pokémon (TV series)',
 'List of Pokémon Trading Card Game sets',
 'Pokémon Ultra Sun and Ultra Moon',
 'Pokémon: The Electric Tale of Pikachu',
 'Ditto (Pokémon)',
 'List of Pokémon',
 'Lists of Nintend

In [9]:
chosen_articles = [articles[i] for i in topics]
chosen_snippets = [snippet_[i] for i in topics]
chosen_articles

['List of Pokémon video games',
 'List of Pokémon episodes (seasons 1–13)',
 'The Pokémon Company',
 'List of generation VII Pokémon',
 'List of generation I Pokémon',
 'List of Pokémon manga',
 'List of generation IV Pokémon',
 'Pokémon Adventures',
 'Pokémon World Championships',
 'Pokémon Diamond and Pearl',
 'Pokémon: Adventures in the Orange Islands',
 'List of generation II Pokémon',
 'Pokémon Crystal',
 'Pokémon: Master Quest',
 'List of Pokémon theme songs',
 'Pokémon Trading Card Game (video game)',
 'Pokémon Pocket Monsters',
 'List of Pokémon characters',
 'Pokémon Red, Blue, and Yellow',
 'Pokémon',
 'List of Pokémon Trading Card Game sets',
 'Pokémon Ultra Sun and Ultra Moon',
 'Pokémon: The Electric Tale of Pikachu']

In [10]:
chosen_full_articles = list(map(lambda x: wiki.page(x, auto_suggest=False).content, chosen_articles))
chosen_articles_images = list(map(lambda x: wiki.page(x, auto_suggest=False).images, chosen_articles))


In [11]:

    
# now its time to store them for RAG
import os


collection_id = client.create_collection(
    name='Articles',
    description='wikipedia articles for presentation',
)

pages = dict(zip(chosen_articles, chosen_full_articles))



## Step 4: Store useful ideas in VectorDB (h2oai)

In [12]:
import re
to_ingest = []
for title, content in pages.items():
    title = re.sub('[\W_]+', '', title)
    name = f"./articles/{title}.txt"
    f = open(name, "w+", encoding="utf-8")
    f.write(content)
    f.close() # dont know why i gotta do this, i think it has to be in binary
    f = open(name, 'rb')
    to_ingest.append(client.upload(name, f))
    print(f"{name} fed!")
    f.close() 

client.ingest_uploads(collection_id, to_ingest)  

./articles/ListofPokémonvideogames.txt fed!
./articles/ListofPokémonepisodesseasons113.txt fed!
./articles/ThePokémonCompany.txt fed!
./articles/ListofgenerationVIIPokémon.txt fed!
./articles/ListofgenerationIPokémon.txt fed!
./articles/ListofPokémonmanga.txt fed!
./articles/ListofgenerationIVPokémon.txt fed!
./articles/PokémonAdventures.txt fed!
./articles/PokémonWorldChampionships.txt fed!
./articles/PokémonDiamondandPearl.txt fed!
./articles/PokémonAdventuresintheOrangeIslands.txt fed!
./articles/ListofgenerationIIPokémon.txt fed!
./articles/PokémonCrystal.txt fed!
./articles/PokémonMasterQuest.txt fed!
./articles/ListofPokémonthemesongs.txt fed!
./articles/PokémonTradingCardGamevideogame.txt fed!
./articles/PokémonPocketMonsters.txt fed!
./articles/ListofPokémoncharacters.txt fed!
./articles/PokémonRedBlueandYellow.txt fed!
./articles/Pokémon.txt fed!
./articles/ListofPokémonTradingCardGamesets.txt fed!
./articles/PokémonUltraSunandUltraMoon.txt fed!
./articles/PokémonTheElectricTa

Job(id='4a17d9e3-b201-44f3-a398-68af58cc8633', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 2, 27, 15, 36, 46, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='6de0bd52efd94606898ea390a1b4b809', status='Indexing done.'), JobStatus(id='1e8c47ea9f974ab5915bf20ce35f2a0c', status='Indexing done.'), JobStatus(id='11b9f1688e07496ea1271685b4a62909', status='Indexing done.'), JobStatus(id='8e6b14e3d9c745168a28821c795ac686', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 2, 27, 15, 38, 41, tzinfo=TzInfo(UTC)), duration='1m55s')

In [13]:
chosen_snippets

['Pokémon (originally "Pocket Monsters") is a series of role-playing video games developed by Game Freak and published by Nintendo and The Pokémon Company.',
 'Pokémon, known in Japan as Pocket Monsters (ポケットモンスター, Poketto Monsutā), is a Japanese anime television series produced by animation studio OLM for TV Tokyo.',
 'The Pokémon Company (株式会社ポケモン, Kabushiki Gaisha Pokemon, TPC) is a Japanese company responsible for brand management, production, publishing, marketing, and licensing of the Pokémon franchise, which consists of video games, a trading card game, anime television series, films, manga, home entertainment products, merchandise, and other ventures.',
 'The seventh generation (Generation VII) of the Pokémon franchise features 86 fictional species of collectible creatures called "Pokémon" introduced to the core video game series in the 2016 Nintendo 3DS games Pokémon Sun and Moon and the 2017 3DS games Pokémon Ultra Sun and Ultra Moon.',
 'The first generation (generation I) o

## Step 5: Plan sections for slide
This is to ensure the entire presentation is a coherent one with a flow/narrative, instead of many disjoint/overlapping generations.
Again, chain of thought prompting is very heavily incorporated

In [14]:
decide_sections = lambda user_query: client.answer_question(
        question=f"""{user_query}
        Please plan the presentation by doing the following:
        1. Explain how you would design the presentation slides such that the presentation will flow well.\
        Remember that each slide must contain something different, and content should not overlap.
        2. Think of a good title for the presentation.
        3. Create a code chunk. Inside that code chunk, generate a JSON array consisting of appropriate slide titles starting from the first slide to the last slide, \
        remembering your answer to point 2. Include the title slide, which is the title for the presentation.

        Below is an example reply. Please adhere strictly to the format in the example below and remember to output the JSON array in a code chunk:  
        1. I would introduce the Transformers franchise and provide general information about its history to ease my viewers into the subject. \
        Then, I will think about subtopics, such as the Transformers films, Transformers characters and Transformers in comics, using the wikipedia entry summaries\
        I have been provided. 
            * For my first subtopic, the Transformers films, I would create two additional slides to expand on Bumblebee (2018) and Revenge of the Fallen (2009)\
            as these are popular films within the franchise. I will order the films chronologically.
            * For my second subtopic on Transformers characters, I will have a slide on the cast of the film. 
            * For my next subtopic...
        2. I think a good title for this presentation is "Transformers: An Overview".
        3. Here is the json array of slide titles:
        ```json
        [
        "Transformers: An Overview", 
        "Introduction to Transformers", 
        "Transformers in Film",
        "Transformers: Revenge of the Fallen (2009)", 
        "Bumblebee (2018)", 
        "Characters in the Transformers Universe", 
        "Transformers in comics",
        "Conclusion"
        ]
        ```
        """,
        system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation.\
        Below is a list of wikipedia entry summaries that are selected for the presentation.\
        You will be asked to come up with slide titles for the presentation. Each line is a 1-sentence summary of a wikipedia page.\
        The number of slides should depend on the amount of information/wikipedia entry summaries available.
        Summaries:
        {chosen_snippets}
        """,
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1' 
    )

all_sections = try_and_parse(user_query, decide_sections, markdown=True)

1. I would start the presentation with an introduction to the Pokémon franchise, providing a brief overview of its history and popularity. Then, I will move on to the different generations of Pokémon, dedicating a slide to each generation and highlighting the unique features and Pokémon species introduced in each. After that, I will discuss the various media through which the Pokémon franchise is presented, such as the anime, manga, and trading card game. I will also include a slide on the Pokémon World Championships, an esports event for the franchise.
2. A good title for the presentation could be "Pokémon: A Deep Dive into the Franchise".
3. Here is the JSON array of slide titles:
```json
[
"Pokémon: A Deep Dive into the Franchise",
"Introduction to Pokémon",
"Generations of Pokémon",
"Generation I: The Original 151",
"Generation II: The Johto Region",
"Generation III: The Hoenn Region",
"Generation IV: The Sinnoh Region",
"Generation V: The Unova Region",
"Generation VI: The Kalos R

In [15]:
sections = all_sections[1:]

sections

['Introduction to Pokémon',
 'Generations of Pokémon',
 'Generation I: The Original 151',
 'Generation II: The Johto Region',
 'Generation III: The Hoenn Region',
 'Generation IV: The Sinnoh Region',
 'Generation V: The Unova Region',
 'Generation VI: The Kalos Region',
 'Generation VII: The Alola Region',
 'Pokémon Media',
 'Pokémon Anime',
 'Pokémon Manga',
 'Pokémon Trading Card Game',
 'Pokémon World Championships',
 'Conclusion']

In [16]:
del client
client = H2OGPTE(
    address="https://h2ogpte.genai.h2o.ai",
    api_key=api
) # does this reset client?

chat_session_id = client.create_chat_session(collection_id)
chat_session_id

'c4930646-184a-4851-8e37-f5484220c214'

#### Ref for slide types:  
0. title and subtitle 
1. title and content 
2. section header 
3. two content 
4. Comparison 
5. Title only  
6. Blank 
7. Content with caption 
8. Pic with caption 


## Step 6: Generate using RAG
LLM chooses colour with chain-of-thought prompting again.

In [17]:
prs = Presentation()
prs.slide_width = Inches(16)
prs.slide_height = Inches(9)
title_slide = prs.slides.add_slide(prs.slide_layouts[0]) 
decide_slide_format = lambda user_query: client.answer_question(
        question=f"""{user_query} Think of a good background colour, in RGB format,\
        for the slides and a good colour, also in RGB format, for the\
        text. Typically, if the text colour is bright (for example RGB [255, 255, 255] is white), then the background colour should be dark
        (RGB [0, 0, 100] is dark blue). Conversely, if the text colour is dark (for example RGB [0, 0, 0] is black), the background colour should be bright\
        . You are free to choose any text and background colour, \
        as long as you follow these rules. Please do not assign grey-scale colours for the text and background (like RGB [50, 50, 50]), as much as possible.

        Explain clearly why you chose the background and text colours. Then, generate a code chunk. Within the code chunk,\
        provide a JSON array containing two colours. Adhere strictly to the example reply below:
        I chose blue RGB [0, 35, 140] for the background color and light yellow RGB [255, 234, 0] for the font color. The contrast makes it easy to read.\
        Furthermore, the colours blue and yellow are associated with the Pokémon Franchise.
        ```
        [{{"background": [0, 0, 140]}}, {{"text": [255, 234, 0]}}]
        ```
        """,
    
        system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation.\
        Here are a list of wikipedia entry summaries that are selected for the presentation:
        {chosen_snippets}
        """,
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1' 
)

format = try_and_parse(user_query, decide_slide_format, markdown=True)

I chose a light blue color RGB [191, 239, 255] for the background color and a dark blue color RGB [0, 38, 166] for the font color. The contrast between the light and dark blue makes it easy to read. Additionally, blue is a color commonly associated with the Pokémon franchise, as it is the color of the iconic Pokéball.
```
[{"background": [191, 239, 255]}, {"text": [0, 38, 166]}]
```


In [18]:
format

[{'background': [191, 239, 255]}, {'text': [0, 38, 166]}]

In [19]:
background = RGBColor(*tuple(list(format[0].values())[0])) 
font = RGBColor(*tuple(list(format[1].values())[0])) 
fill = title_slide.background.fill
fill.solid()
fill.fore_color.rgb = background


title_slide.shapes.title.text = all_sections[0]
title_slide.shapes.title.text_frame.paragraphs[0].font.color.rgb =  font
title_slide.shapes.title.text_frame.paragraphs[0].font.name = 'Montserrat'
title_slide.shapes.title.text_frame.paragraphs[0].font.bold = True

first_shape =  title_slide.shapes[0]
first_shape.left, first_shape.top, first_shape.width, first_shape.height = (prs.slide_width - Inches(12))//2, \
(prs.slide_height-first_shape.height)//2 - Inches(1),\
Inches(12),\
Inches(2)

In [20]:

with client.connect(chat_session_id) as session:

    for section in tqdm(sections):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        fill = slide.background.fill
        fill.solid()
        fill.fore_color.rgb = background

        
        contents = slide.placeholders[1]
        contents.text_frame.word_wrap = True

        title = slide.shapes.title
        title.text = section
        title.text_frame.paragraphs[0].font.color.rgb = font
        title.text_frame.paragraphs[0].font.size = Pt(32)
        title.text_frame.paragraphs[0].font.name = 'Karla'
       
       
        content = session.query(
            
            message = section,
            system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation. \
            The slides of the presentation are as follows: {sections}
            You are now tasked with generating the content of one slide, which will be provided by the user.
            """,
            pre_prompt_query="You have been provided with the following information, which may be useful in your task.",
            prompt_query="""Decide if the information is relevant, and use it if needed.\
            Generate the content required in the slide provided by the user. You only need to generate the contents of the slide, not the title\
            or anything else. Remember, you are designing one slide, so do not generate long paragraphs. Instead, summarise your most important points\
            in up to 3 sentences each, and do not generate more than 15 lines of text. Between each point, leave a line.

            Here is an example. Please adhere to this example strictly:
            This is the first point I am trying to make. I will explain this first point in the second sentence.
            
            This is the second point I am trying to make. This is the second sentence for the second point. \
            I need a third sentence to fully explain this point. Notice that I will leave a line for the next point.

            This is the third point.
            """,
            llm="mistralai/Mixtral-8x7B-Instruct-v0.1",
            rag_config={
                "rag_type": "rag",
            },
        ).content

        contents.text = content
        
        for paragraph in contents.text_frame.paragraphs:
            paragraph.space_after = 1
            paragraph.space_before = 1
            # paragraph.level = 0
           
            paragraph.font.size = Pt(22)  
            paragraph.font.color.rgb = font
            paragraph.font.name = 'Karla'

        contents.text_frame.auto_size = MSO_AUTO_SIZE.SHAPE_TO_FIT_TEXT
        shapes = slide.shapes
        new_width = Inches(14)
        new_height = Inches(7)
        shapes[0].height, shapes[0].width, shapes[0].top, shapes[0].left = shapes[0].height, new_width, shapes[0].top, (prs.slide_width-new_width)//2
        shapes[1].height, shapes[1].width, shapes[1].top, shapes[1].left = new_height, new_width, shapes[1].top, (prs.slide_width-new_width)//2
        
        

# gpt-4-1106-preview


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [19:07<00:00, 76.49s/it]


## Step 7: Enjoy

In [21]:
sanitised = re.sub(r'[\W_]+', '_', all_sections[0])
prs.save(f"./presentations/{sanitised}.pptx")


## Appendix: Extra Code that may be useful in the future
```python
# Create a chat session
# chat_session_id = client.create_chat_session(collection_id)

# # Query the collection
# with client.connect(chat_session_id) as session:
#     reply = session.query(
#         'How many paper clips were shipped to Scranton?',
#         llm="gpt-4-0613"
#     )
#     print(reply.content)

#     reply = session.query(
#         'Did David Brent co-sign the contract with Initech?',
#         timeout=60,
#         llm="gpt-4-0613"
#     )
#     print(reply.content)

# # Summarize each document
# documents = client.list_documents_in_collection(collection_id, offset=0, limit=99)
# for doc in documents:
#     summary = client.summarize_document(
#         document_id=doc.id,
#         timeout=60,
#     )
#     print(summary.content)


#client.delete_documents_from_collection
```